In [1]:
import asyncio
from _getEtf_YenTa import main

await main()

In [ ]:
import os
import pandas as pd

# 設置檔案路徑
file_path = './data/etf/00940/'

# 獲取所有符合模式的文件名稱
files = [f for f in os.listdir(file_path) if f.endswith('.csv') and f.startswith('2024')]

# 初始化一個空的DataFrame來存儲結果
result = pd.DataFrame()

from datetime import datetime #, timedelta
todayFile = datetime.today().strftime("%Y%m%d%H:%M") # 取得今天的日期
todayFile += "比較.csv"

for file in files:
    # 讀取CSV文件
    df = pd.read_csv(os.path.join(file_path, file), header=None, names=['代碼', '名稱', '數量', '權重'])
    
    # 計算每行數據與前一行的差值
    if result.empty:
        result = df.copy()
    else:
        result['數量'] = df['數量'] - result['數量']
        result['權重'] = df['權重'] - result['權重']
        # 將結果寫入CSV文件
    result.to_csv(f'{file_path}/{todayFile}', index=False)

In [ ]:
import pandas as pd
import os
import re

# 設置 CSV 檔案目錄
csv_dir = 'data/00940'

# 初始化一個空的 DataFrame
merged_data = pd.DataFrame()

# 讀取每個 CSV 檔案並合併
for file in os.listdir(csv_dir):
    if file.endswith('.csv'):  # 排除包含 2024 年份的檔案
        # 使用正則表達式提取檔案名稱中的 MMDD
        match = re.search(r'(\d{4})(\d{2})(\d{2})', file)
        if match:
            mmdd = match.group(2) + match.group(3)
            file_path = os.path.join(csv_dir, file)
            df = pd.read_csv(file_path, delimiter='\t', header=None, skiprows=[0], names=['代碼', '名稱', '數量', '權重'])
            df.set_index(['代碼', '名稱'], inplace=True)
            merged_data[mmdd] = df['數量']

# 創建一個新的欄位存儲最後兩個數量的差值
merged_data['差值'] = merged_data.iloc[:, -1] - merged_data.iloc[:, -2]

# 按照差值由大到小排列整個 DataFrame
sorted_data = merged_data.sort_values(by='差值', ascending=False)

# 刪除 'V' 欄位
# sorted_data.drop(columns=['V'], inplace=True)
# 顯示整理後的資料
# print(sorted_data)

from datetime import datetime #, timedelta
todayFile = datetime.today().strftime("%Y%m%d%H")                 # 取得今天的日期
df = pd.DataFrame(sorted_data) # 将字典列表转换为 DataFrame
df.to_csv(f"data/compared/00940_{todayFile}.csv", index=True, sep='\t') #儲存成CSV

